In [1]:
import json
from tqdm.autonotebook import tqdm
from evaluate_metrics import compute_f1, compute_avg_bleu

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
with open('./data/test/evjvqa_public_test-lang-qtype-answer.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [3]:
test_annotations = test_data['annotations']

qid2lang = {}
qid2qtype = {}

for anno in tqdm(test_annotations):
    qid2lang[str(anno['id'])] = anno['language']
    qid2qtype[str(anno['id'])] = anno['question_type']
    
len(test_annotations), len(qid2lang), len(qid2qtype)

  0%|          | 0/5015 [00:00<?, ?it/s]

(5015, 5015, 5015)

In [4]:
with open('./outputs/groundtruth_results.json', 'r', encoding='utf-8') as f:
    gt_results = json.load(f)

## MT5 + VIT

In [25]:
with open('./outputs/results-mt5-vit.json', 'r', encoding='utf-8') as f:
    mt5vit_results = json.load(f)

sep_qtype = False
languages = ['en', 'vi', 'ja']
# question_types = list(set(qid2qtype.values()))
question_types = ['HOW_MANY', 'WHAT_COLOR', 'WHERE', 'WHO', 'HOW', 'WHAT_IS', 'WHAT_DO', 'WHICH', 'OTHERS']

for lang in languages:
    pred = {}
    grth = {}
    print(f'------------------- {lang.upper()} -------------------')
    if sep_qtype:
        for qtype in question_types:
            pred = {}
            grth = {}
            for k, v in mt5vit_results.items():
                if qid2lang[k] == lang and qid2qtype[k] == qtype:
                    pred[k] = v
                    grth[k] = gt_results[k]
            
            f1 = compute_f1(a_gold=grth, a_pred=pred)
            bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
            print(f"Metrics of Language={lang} - Question Type={qtype}: F1 = {f1} and Bleu = {bleu}")
    
        print(f'------------------- END -------------------')
    else:
        for k, v in mt5vit_results.items():
            if qid2lang[k] == lang:
                pred[k] = v
                grth[k] = gt_results[k]

        f1 = compute_f1(a_gold=grth, a_pred=pred)
        bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
        print(f"Metrics of Language={lang}: F1 = {f1} and Bleu = {bleu}")

------------------- EN -------------------
Metrics of Language=en: F1 = 0.18036668552330218 and Bleu = 0.12762038026689043
------------------- VI -------------------
Metrics of Language=vi: F1 = 0.2697203135337372 and Bleu = 0.1965703409687105
------------------- JA -------------------
Metrics of Language=ja: F1 = 0.28681220185811296 and Bleu = 0.22247196627623678


## MT5 + CLIP

In [27]:
with open('./outputs/results-mt5-clip.json', 'r', encoding='utf-8') as f:
    mt5vit_results = json.load(f)

sep_qtype = True
languages = ['en', 'vi', 'ja']
# question_types = list(set(qid2qtype.values()))
question_types = ['HOW_MANY', 'WHAT_COLOR', 'WHERE', 'WHO', 'HOW', 'WHAT_IS', 'WHAT_DO', 'WHICH', 'OTHERS']

for lang in languages:
    pred = {}
    grth = {}
    print(f'------------------- {lang.upper()} -------------------')
    if sep_qtype:
        for qtype in question_types:
            pred = {}
            grth = {}
            for k, v in mt5vit_results.items():
                if qid2lang[k] == lang and qid2qtype[k] == qtype:
                    pred[k] = v
                    grth[k] = gt_results[k]
            
            f1 = compute_f1(a_gold=grth, a_pred=pred)
            bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
            print(f"Metrics of Language={lang} - Question Type={qtype}: F1 = {f1} and Bleu = {bleu}")
    
        print(f'------------------- END -------------------')
    else:
        for k, v in mt5vit_results.items():
            if qid2lang[k] == lang:
                pred[k] = v
                grth[k] = gt_results[k]

        f1 = compute_f1(a_gold=grth, a_pred=pred)
        bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
        print(f"Metrics of Language={lang}: F1 = {f1} and Bleu = {bleu}")

------------------- EN -------------------
Metrics of Language=en - Question Type=HOW_MANY: F1 = 0.13339016958487496 and Bleu = 0.15230238711372088
Metrics of Language=en - Question Type=WHAT_COLOR: F1 = 0.18867264810661036 and Bleu = 0.11885855015466021
Metrics of Language=en - Question Type=WHERE: F1 = 0.21169268100878802 and Bleu = 0.10593184382072081
Metrics of Language=en - Question Type=WHO: F1 = 0.1892021740442793 and Bleu = 0.10434442536656838
Metrics of Language=en - Question Type=HOW: F1 = 0.12821011168762037 and Bleu = 0.05377825721395391
Metrics of Language=en - Question Type=WHAT_IS: F1 = 0.13199692708824523 and Bleu = 0.07882378140220024
Metrics of Language=en - Question Type=WHAT_DO: F1 = 0.10929560189096035 and Bleu = 0.042158015981437355
Metrics of Language=en - Question Type=WHICH: F1 = 0.33082651111143757 and Bleu = 0.27822888234924353
Metrics of Language=en - Question Type=OTHERS: F1 = 0.1278827139938251 and Bleu = 0.07079791924006913
------------------- END -------

/home/phucpx/vinbdi/EVJVQA/evaluate_metrics.py:142: RuntimeWarning: Mean of empty slice.
  return np.array(scores).mean()
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## MT5 + DETR

## MT5 + BEiT

In [5]:
with open('./outputs/results-beit-last.json', 'r', encoding='utf-8') as f:
    mt5beit_results = json.load(f)

sep_qtype = True
languages = ['en', 'vi', 'ja']
# question_types = list(set(qid2qtype.values()))
question_types = ['HOW_MANY', 'WHAT_COLOR', 'WHERE', 'WHO', 'HOW', 'WHAT_IS', 'WHAT_DO', 'WHICH', 'OTHERS']

for lang in languages:
    pred = {}
    grth = {}
    print(f'------------------- {lang.upper()} -------------------')
    if sep_qtype:
        for qtype in question_types:
            pred = {}
            grth = {}
            for k, v in mt5beit_results.items():
                if qid2lang[k] == lang and qid2qtype[k] == qtype:
                    pred[k] = v
                    grth[k] = gt_results[k]
            
            f1 = compute_f1(a_gold=grth, a_pred=pred)
            bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
            print(f"Metrics of Language={lang} - Question Type={qtype}: F1 = {f1} and Bleu = {bleu}")
    
        print(f'------------------- END -------------------')
    else:
        for k, v in mt5beit_results.items():
            if qid2lang[k] == lang:
                pred[k] = v
                grth[k] = gt_results[k]

        f1 = compute_f1(a_gold=grth, a_pred=pred)
        bleu = compute_avg_bleu(a_gold=grth, a_pred=pred)
            
        print(f"Metrics of Language={lang}: F1 = {f1} and Bleu = {bleu}")

------------------- EN -------------------
Metrics of Language=en - Question Type=HOW_MANY: F1 = 0.13053295067253193 and Bleu = 0.1470569420457975
Metrics of Language=en - Question Type=WHAT_COLOR: F1 = 0.20336276899817407 and Bleu = 0.12395571180106006
Metrics of Language=en - Question Type=WHERE: F1 = 0.17593017702369135 and Bleu = 0.0746543972312971
Metrics of Language=en - Question Type=WHO: F1 = 0.1871823731823732 and Bleu = 0.08006352108680244
Metrics of Language=en - Question Type=HOW: F1 = 0.21282214517508635 and Bleu = 0.05022458593174603
Metrics of Language=en - Question Type=WHAT_IS: F1 = 0.1307706030307934 and Bleu = 0.06293572549477212
Metrics of Language=en - Question Type=WHAT_DO: F1 = 0.10428357449841824 and Bleu = 0.03382330264840466
Metrics of Language=en - Question Type=WHICH: F1 = 0.24861698557398054 and Bleu = 0.1210749153495035
Metrics of Language=en - Question Type=OTHERS: F1 = 0.12334489070600181 and Bleu = 0.06517309386845564
------------------- END -----------

/home/phucpx/vinbdi/EVJVQA/evaluate_metrics.py:142: RuntimeWarning: Mean of empty slice.
  return np.array(scores).mean()
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Prepare training data

In [8]:
with open('./data/train/evjvqa_train_lang_qtype-detailed.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
    
train_annotations = train_data['annotations']

In [9]:
subtrain_annotations = []

LIST_QTYPES = ['WHAT_IS', 'WHO', 'WHERE', 'HOW_MANY']

for anno in tqdm(train_annotations):
    if anno['question_type'] in LIST_QTYPES:
        subtrain_annotations.append(anno)
        
print(len(subtrain_annotations))

subtrain_dict = {
    'images': train_data['images'],
    'annotations': subtrain_annotations
}

  0%|          | 0/23785 [00:00<?, ?it/s]

12121


In [10]:
with open('./data/train/evjvqa-subtrain-obj.json', 'w', encoding='utf-8') as f:
    json.dump(subtrain_dict, f, indent=4, ensure_ascii=False)

In [6]:
with open('./data/test/evjvqa_public_test-lang-qtype-answer.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_annotations = test_data['annotations']

subtest_annotations = []

LIST_QTYPES = ['WHAT_IS', 'WHO', 'WHERE', 'HOW_MANY']

for anno in tqdm(test_annotations):
    if anno['question_type'] in LIST_QTYPES:
        subtest_annotations.append(anno)
        
print(len(subtest_annotations))

subtest_dict = {
    'images': test_data['images'],
    'annotations': subtest_annotations
}

with open('./data/test/evjvqa-subtest-obj.json', 'w', encoding='utf-8') as f:
    json.dump(subtest_dict, f, indent=4, ensure_ascii=False)

  0%|          | 0/5015 [00:00<?, ?it/s]

2566
